In [1]:
import pandas as pd

clinical_details = pd.read_parquet("genomic_data_parquet/clinical_Details.parquet")
clinical_details

,cancerSiteIdcCode,cancerSite,cancerLocationIdcCode,cancerLocation,morphologyIdcCode,morphology,laterality,patientID,patientId,patientUUId,...,bloodThinningMedicationSpecify,isMedicalAllergies,medicalAllergiesSpecify,immunosupression,imunosupressionSpecify,otherFormsOfTobaccoUse,otherFormsOfTobaccoUseSpecify,cancerDetailsComments,diseaseStatus,dateOfDiagnosis
0,C24,Other and unspecified biliary tract,C24.8,Overlapping lesion of biliary tract,8163/3,Pancreatobiliary-type carcinoma,Unknown,IN-423-VEJA,IN-423-VEJA,65a44a2c-ea7d-4ad6-a8d4-41b2381a7513,...,None,False,None,Unknown,None,Unknown,None,None,None,None
1,C25,Pancreas,C25.0,Head of pancreas,8140/3,"Adenocarcinoma, NOS",Not a paired Site,IN-423-VERI,IN-423-VERI,08fc448e-42c9-4562-8160-2a64ad62fa10,...,None,None,None,None,None,None,None,None,None,None
2,C75,Other endocrine glands,C75.0,Parathyroid gland,8255/3,Adenocarcinoma with mixed subtypes,Right,IN-423-VHDD,IN-423-VHDD,69c003fc-42fe-4bcb-ba69-8124a51be0e6,...,None,None,None,None,None,None,None,None,None,None
3,C34,Lung,C34.9,Lung NOS,8140/3,"Adenocarcinoma, NOS",Unknown,IN-423-VKQA,IN-423-VKQA,1e27564b-72df-489f-8fbc-dfced669a616,...,None,None,None,None,None,None,None,None,None,None
4,C34,Lung,C34.9,Lung NOS,8070/3,"Squamous cell carcinoma, NOS",Unknown,IN-423-W4XB,IN-423-W4XB,45ae2fbf-586d-4aaf-b890-223ddc7fcf3f,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,C34,Lung,C34.9,Lung NOS,8046/3,Non-small cell carcinoma,Right,IN-423-XCZAA,IN-423-XCZAA,fc693281-301e-4281-944f-6149d8a08212,...,None,False,None,Unknown,None,Unknown,None,None,None,2023-02-25
90,C06,Other and unspecified parts of mouth,C06.0,Cheek mucosa,8200/3,Adenoid cystic carcinoma,Right,IN-423-XHAAN,IN-423-XHAAN,997b81c8-2fed-40a6-beef-41e322b5bf4b,...,None,None,None,Unknown,None,Never,None,None,None,2023-06-01
91,C16,Stomach,C16.3,Gastric antrum,8144/3,"Adenocarcinoma, intestinal type",Not a paired Site,IN-423-XK4AC,IN-423-XK4AC,21594e5f-a673-4965-8f5d-a20b4f55ae6d,...,None,None,None,Unknown,None,Never,None,None,None,2024-07-12
92,C49,"Connective, subcutaneous and other soft tissues",C49.3,"Connective tissue thorax (excludes Thymus, Hea...",8000/3,"Neoplasm, malignant",Not a paired Site,IN-423-XLJAO,IN-423-XLJAO,484a9672-7211-4c40-ad45-2dd9c0d725dc,...,None,None,None,Unknown,None,Unknown,None,paravertebral,None,None


In [4]:
import redis
import sys

# Define your connection parameters
REDIS_HOST = 'localhost'
REDIS_PORT = 6379 # Keep this at 6100 for now, as that's what your error implies you're trying

# --- Method 1: Direct instantiation ---
print("--- Attempting connection via direct parameters ---")
try:
    r_direct = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, db=0)

    # Correct way to get host and port for direct instantiation
    # They are typically stored in the ConnectionPool's connection_kwargs
    configured_host_direct = r_direct.connection_pool.connection_kwargs.get('host')
    configured_port_direct = r_direct.connection_pool.connection_kwargs.get('port')

    print(f"Configured Host (direct): {configured_host_direct}, Configured Port (direct): {configured_port_direct}")

    r_direct.ping()
    print(f"SUCCESS: Connected via direct parameters to {configured_host_direct}:{configured_port_direct}")

except redis.exceptions.ConnectionError as e:
    print(f"ERROR (direct connection): Redis connection failed: {e}")
    print(f"Attempted connection to: {REDIS_HOST}:{REDIS_PORT}")
    print("Please check server status, port, firewall, and redis.conf 'bind' setting.")
    # In a real application, you might exit here or try alternative configurations
    # sys.exit(1) # Uncomment to exit on first error

except Exception as e:
    print(f"An unexpected error occurred with direct connection: {e}")
    # sys.exit(1) # Uncomment to exit on first error

print("\n--- Attempting connection via from_url ---")
# --- Method 2: from_url instantiation ---
redis_url = f"redis://{REDIS_HOST}:{REDIS_PORT}/0"
try:
    r_url = redis.from_url(redis_url)

    # Correct way to get host and port when using from_url
    # The URL itself is a good indicator, but you can also parse connection_kwargs
    configured_host_url = r_url.connection_pool.connection_kwargs.get('host')
    configured_port_url = r_url.connection_pool.connection_kwargs.get('port')

    print(f"Configured URL: {redis_url}")
    print(f"Resolved Host (from_url): {configured_host_url}, Resolved Port (from_url): {configured_port_url}")

    r_url.ping()
    print(f"SUCCESS: Connected via from_url to {configured_host_url}:{configured_port_url}")

except redis.exceptions.ConnectionError as e:
    print(f"ERROR (from_url connection): Redis connection failed: {e}")
    print(f"Attempted connection to: {REDIS_HOST}:{REDIS_PORT} using URL: {redis_url}")
    print("Please check server status, port, firewall, and redis.conf 'bind' setting.")
    sys.exit(1) # Exit if this fails, as it's typically the main method

except Exception as e:
    print(f"An unexpected error occurred with from_url connection: {e}")
    sys.exit(1)

print("\n--- Successful connection(s) established. ---")
# You can now use r_direct or r_url for your Redis operations

--- Attempting connection via direct parameters ---
Configured Host (direct): localhost, Configured Port (direct): 6379
ERROR (direct connection): Redis connection failed: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.
Attempted connection to: localhost:6379
Please check server status, port, firewall, and redis.conf 'bind' setting.

--- Attempting connection via from_url ---
Configured URL: redis://localhost:6379/0
Resolved Host (from_url): localhost, Resolved Port (from_url): 6379
ERROR (from_url connection): Redis connection failed: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.
Attempted connection to: localhost:6379 using URL: redis://localhost:6379/0
Please check server status, port, firewall, and redis.conf 'bind' setting.


SystemExit: 1

c:\Users\satya\OneDrive\Documents\oncotwin_Tfirst8\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
